In [1]:
import datetime

from google.cloud import bigquery
import pandas as pd
import pytz
from google.oauth2 import service_account
import numpy as np

In [2]:
credentials = service_account.Credentials.from_service_account_file('pax-11-366616-3e921bf3ac9e.json')

In [3]:
client = bigquery.Client(credentials=credentials)

In [23]:
!pip3 install pymysql

In [24]:
import pymysql

In [27]:

# Connect to the database
connection = pymysql.connect(host='34.148.88.4',
                         user='root',
                         password='tejaswini',
                         db='oltp_schema')

# create cursor
cursor=connection.cursor()

In [28]:
df_order_details=pd.read_sql(f'''select * from order_details''',connection)
df_consumer_master=pd.read_sql(f'''select * from customer_master''',connection)
df_order_items=pd.read_sql(f'''select * from order_items''',connection)
df_product_master=pd.read_sql(f'''select * from product_master''',connection)

C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 c

In [29]:
def insert_dim_order(df_order_details):
    g = df_order_details
    #g = df_order_details.groupby("orderid").tail(1)
    x = g[["orderid","order_status_update_timestamp","order_status"]]
    dim_order = x.reset_index()
    del dim_order["index"]
    return dim_order

In [30]:
def insert_dim_customer(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_customer =df_consumer_master[["customerid","name","address_id"]]
    dim_customer["start_date"]= (df_consumer_master['update_timestamp'].dt.date).astype('datetime64')
    dim_customer["end_date"] = np.nan
    dim_customer["end_date"]=pd.to_datetime(dim_customer["end_date"])
    return dim_customer

In [31]:
def insert_dim_address(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_address = df_consumer_master[["address_id","address","city","state","pincode"]]
    return dim_address

In [32]:
def insert_dim_product(df_product_master):
    dim_product = df_product_master
    dim_product["start_date"]=np.nan
    dim_product["start_date"]=pd.to_datetime(dim_product["start_date"])
    dim_product["end_date"]=np.nan
    dim_product["end_date"]=pd.to_datetime(dim_product["end_date"])
    return dim_product

In [33]:
def insert_f_order_details(df_order_details,df_order_items):
    f_order_details = pd.merge(df_order_details.groupby("orderid").tail(1), df_order_items, how='inner')[["orderid","order_status_update_timestamp","productid","quantity"]]
    f_order_details.columns = ["orderid","order_delivery_timestamp","productid","quantity"]
    return f_order_details

In [34]:
def insert_fact_daily_orders(df_consumer_master,df_order_details,df_order_items):
    x=df_order_details.groupby("orderid").head(1)[["customerid","orderid","order_status_update_timestamp"]].reset_index()
    del x["index"]
    y=df_order_details.groupby("orderid").tail(1)["order_status_update_timestamp"].reset_index()
    del y["index"]     
    fact_daily_orders = pd.concat([x,y],axis=1)
    fact_daily_orders.columns = ["customerid","orderid","order_received_timestamp","order_delivery_timestamp"]
    l=[]
    for i in fact_daily_orders["customerid"]:
             l.append(int(df_consumer_master.where(df_consumer_master["customerid"]==i).dropna()["pincode"]))
            # l is pincode column
    fact_daily_orders["pincode"] = l
        
    
    k=[]
    for i in df_order_items["productid"]:
        k.append(int(df_product_master.where(df_product_master["productid"]==i).dropna()["rate"] * df_order_items.iloc[i,2] ))    
    df_order_items["Total"]=k    
    k=[]
    k = df_order_items.groupby('orderid').sum()["Total"]
    k1 = df_order_items.groupby('orderid').sum()["quantity"]
    fact_daily_orders["order_amount"]=list(k)
    fact_daily_orders["item_count"]=list(k1)
    fact_daily_orders["order_delivery_time_seconds"] =fact_daily_orders["order_delivery_timestamp"] - fact_daily_orders["order_received_timestamp"]
    return fact_daily_orders

In [57]:
def insert_fact_daily_orders(df_consumer_master,df_order_details,df_order_items):
    x=df_order_details.groupby("orderid").head(1)[["customerid","orderid","order_status_update_timestamp"]].reset_index()
    del x["index"]
    y=df_order_details.groupby("orderid").tail(1)["order_status_update_timestamp"].reset_index()
    del y["index"]     
    fact_daily_orders = pd.concat([x,y],axis=1)
    fact_daily_orders.columns = ["customerid","orderid","order_received_timestamp","order_delivery_timestamp"]
    l=[]
    for i in fact_daily_orders["customerid"]:
             l.append(int(df_consumer_master.where(df_consumer_master["customerid"]==i).dropna()["pincode"]))
            # l is pincode column
    fact_daily_orders["pincode"] = l
        
    k=[]
    df=pd.DataFrame()
    for i in df_order_items["productid"]:
        df['price']['i]=int(df_product_master.where(df_product_master["productid"]==i).dropna()["rate"] * df_order_items.iloc[i,2] )
    df['price']=list(k)
    return df

In [35]:
dim_order = insert_dim_order(df_order_details)

In [46]:
df_order_items.head()

,orderid,productid,quantity
0,1,11,4
1,1,17,2
2,1,26,2
3,1,50,1
4,2,16,2


In [48]:
df_order_items.iloc[0,2]

4

In [36]:
dim_customer = insert_dim_customer(df_consumer_master)

C:\Users\dhara\AppData\Local\Temp\ipykernel_11632\43446549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_customer["start_date"]= (df_consumer_master['update_timestamp'].dt.date).astype('datetime64')
C:\Users\dhara\AppData\Local\Temp\ipykernel_11632\43446549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_customer["end_date"] = np.nan
C:\Users\dhara\AppData\Local\Temp\ipykernel_11632\43446549.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [37]:
dim_address = insert_dim_address(df_consumer_master)

In [38]:
dim_product=insert_dim_product(df_product_master)

In [39]:
f_order_details = insert_f_order_details(df_order_details,df_order_items)


In [65]:
fact_daily_orders = insert_fact_daily_orders(df_consumer_master,df_order_details,df_order_items)


KeyboardInterrupt



In [59]:
tableRef1 = client.dataset("star_schema").table("dim_order")
client.load_table_from_dataframe(dim_order,tableRef1)

LoadJob<project=pax-11-366616, location=US, id=7f85af4b-4d7d-4fd0-8bcf-6c56b5af5ba2>

In [60]:
tableRef2 = client.dataset("star_schema").table("dim_customer")
client.load_table_from_dataframe(dim_customer,tableRef2)

LoadJob<project=pax-11-366616, location=US, id=74f61190-b72d-4232-9496-b5634debb38e>

In [61]:
tableRef3 = client.dataset("star_schema").table("dim_address")
client.load_table_from_dataframe(dim_address,tableRef3)


LoadJob<project=pax-11-366616, location=US, id=386c80aa-6851-411c-bf53-e30892915bcc>

In [62]:
tableRef4 = client.dataset("star_schema").table("dim_product")
client.load_table_from_dataframe(dim_product,tableRef4)

LoadJob<project=pax-11-366616, location=US, id=570d24f1-0aad-4ae7-9742-d8201d3ebaee>

In [63]:
tableRef5 = client.dataset("star_schema").table("f_order_details")
client.load_table_from_dataframe(f_order_details,tableRef5)

LoadJob<project=pax-11-366616, location=US, id=21b05df4-779d-45e7-917f-e9edf5769b65>

In [ ]:
tableRef6 = client.dataset("star_schema").table("fact_daily_orders")
client.load_table_from_dataframe(fact_daily_orders,tableRef6)